In [1]:
import os
import cv2
import numpy as np

In [2]:
data = []
CATEGORIES = ["negatives", "positives"]

for category in CATEGORIES:
    path = category
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img))
        
        data.append([img_array, class_num])
       

In [21]:
list_red_mean = []
list_green_mean = []
list_blue_mean = []
list_red_max = []
list_blue_max = []

new_data = []
for img, num in data:
    red_mean = 0
    green_mean = 0
    blue_mean = 0
    red_max = 0
    blue_max = 0
    green_min = 255
    red_min = 255
    for line in img:
        for pixel in line:
            red_mean += pixel[0]*(1/(img.shape[0]*img.shape[1]))
            green_mean += pixel[1]*(1/(img.shape[0]*img.shape[1]))
            blue_mean += pixel[2]*(1/(img.shape[0]*img.shape[1]))
            if pixel[0] > red_max:    
                red_max = pixel[0]
            if pixel[2] > blue_max:
                blue_max = pixel[2]
            if pixel[1] < green_min:
                green_min = pixel[1]
            if pixel[0] < red_min:
                red_min = pixel[0]    
    list_red_mean.append(red_mean)
    list_green_mean.append(green_mean)
    list_blue_mean.append(blue_mean)
    list_red_max.append(red_max)
    list_blue_max.append(blue_max)
    new_data.append([green_mean, blue_mean, red_max, blue_max, red_min, green_min, num])
    #new_data.append([red_mean, green_mean, blue_mean, red_max, blue_max, red_min, green_min, num])
    
num_features = len(new_data[0][-2:-len(new_data[0])-1:-1])

In [22]:
mü_0 = np.zeros((num_features,1))
mü_1 = np.zeros((num_features,1))
phi = 0
gamma = np.zeros((num_features,num_features))

for vec in new_data:
    phi += (1/len(new_data))*vec[num_features]
    if vec[num_features] == 0:    
        mü_0 += (1/30)*np.reshape(vec[0:num_features], (num_features,1))
    else:    
        mü_1 += (1/30)*np.reshape(vec[0:num_features], (num_features,1))
        
for vec in new_data:
    if vec[num_features] == 0:
        gamma += (1/60)*(np.reshape(vec[0:num_features], (num_features,1))-mü_0)@np.transpose(np.reshape(vec[0:num_features], (num_features,1))-mü_0)
    else:
        gamma += (1/60)*(np.reshape(vec[0:num_features], (num_features,1))-mü_1)@np.transpose(np.reshape(vec[0:num_features], (num_features,1))-mü_1)

# Parameters

In [25]:
print(mü_0)
print(mü_1)
print(gamma)

[[126.82800926]
 [149.96666667]
 [187.03333333]
 [103.4       ]
 [102.8       ]]
[[139.52424769]
 [165.33333333]
 [209.03333333]
 [104.73333333]
 [ 47.16666667]]
[[252.26656152 132.13186535 109.60448785 170.19188465 176.06197242]
 [132.13186535 125.86055556 102.995       99.61777778  70.63555556]
 [109.60448785 102.995      121.16555556  67.59777778  48.43388889]
 [170.19188465  99.61777778  67.59777778 186.98444444 133.87888889]
 [176.06197242  70.63555556  48.43388889 133.87888889 221.08277778]]


In [23]:
def classify(x, mü_0, mü_1, gamma, phi):
    x = np.reshape(x, (len(x),1))
    p_x0 = (1/(2*(np.pi**(len(x)/2))*np.linalg.det(gamma)**(1/2)))*np.exp(-(1/2)*(np.transpose(x-mü_0)@np.linalg.inv(gamma))@(x-mü_0))
    p_x1 = (1/(2*(np.pi**(len(x)/2))*np.linalg.det(gamma)**(1/2)))*np.exp(-(1/2)*(np.transpose(x-mü_1)@np.linalg.inv(gamma))@(x-mü_1))    
    p_0 = (p_x0*(1-phi))/(p_x0*(1-phi)+p_x1*phi)
    p_1 = (p_x1*phi)/(p_x0*(1-phi)+p_x1*phi)
    if p_0 > p_1:
        return 0
    else:
        return 1

In [24]:
import random
k = 0
random.shuffle(new_data)
for dat in new_data:
    k += 1
    print(classify(dat[0:num_features], mü_0, mü_1, gamma, phi)-dat[-1])
    

0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [116]:
print(mü_0.round(1))
print(mü_1.round(1))
print(gamma.round(1))



[[127.3]
 [126.8]
 [163.7]
 [150. ]
 [187. ]
 [103.4]
 [102.8]]
[[142.9]
 [139.5]
 [176.2]
 [165.3]
 [209. ]
 [104.7]
 [ 47.2]]
[[131.  170.5 139.3 111.5  91.5 128.6 113. ]
 [170.5 252.3 192.7 132.1 109.6 170.2 176.1]
 [139.3 192.7 172.5 113.7 108.6 129.  118.7]
 [111.5 132.1 113.7 125.9 103.   99.6  70.6]
 [ 91.5 109.6 108.6 103.  121.2  67.6  48.4]
 [128.6 170.2 129.   99.6  67.6 187.  133.9]
 [113.  176.1 118.7  70.6  48.4 133.9 221.1]]
